In [1]:
#-*-coding:utf8-*-
# step1: data processing
# step2: loading machine learning model and predict users' behaviour
#need to be added the condition for NaN

import pandas as pd
import string, re, collections
from collections import namedtuple
import numpy as np
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib


# check if the feature includes words in our words_library

def ads_search(obj_content):
    if obj_content:
        file_obj = open('/home/wilson/Desktop/databases/words_library.txt', 'r')
        words_library = file_obj.read()
        ads_library = re.compile(words_library, re.I)
#         print(obj_content)
        result = ads_library.findall(obj_content)
        if result:
            
            return 0 # exist ads return 0
        else:
            return 1
    else:
        return 1


# check if the feature includes phone numbers
def phone_search(feature_s):
    if feature_s:
        phone_re = re.compile('(\d{3}\D{0,1}\d{3}\D{0,1}\d{4})')
        phone_label = phone_re.findall(feature_s)
        if phone_label:
            return 0# contains 0
        else:
            #         print("This sentence doesn't contain phone num")
            return 1
    else:
        return 1


# count the length of a string
def string_len(sentence_in):
    count_en = count_dg = count_sp = count_zh = count_pu = 0
    if sentence_in:
        s_leng = len(sentence_in)
        for one in sentence_in:
            if one in string.ascii_letters:
                count_en += 1
            elif one.isdigit():
                count_dg += 1
            elif one.isspace():
                count_sp += 1
            elif one.isalpha():
                count_zh += 1
            else:
                count_pu += 1

        total_chars = count_en + count_dg + count_sp + count_zh + count_pu
        if total_chars == s_leng:
            return (count_en, count_dg, count_sp, count_zh, count_pu)
        else:
            print('String length counting is wrong')
            return None
    else:
        return (count_en, count_dg, count_sp, count_zh, count_pu)


# count the num of english words in one signature
def count_words(sentence_in):
    if sentence_in:
        word_find = re.compile('[a-zA-Z0-9]+')
        counter = collections.Counter(word_find.findall(sentence_in))
        kind_words = len(counter)
        num_words = sum(counter.values())
        return (kind_words, num_words)
    else:
        return(0,0)

def apply_and_concat(dataframe, field, func, column_names):
    return pd.concat((
        dataframe,
        dataframe[field].apply(
            lambda cell: pd.Series(func(cell), index=column_names))), axis=1)



if __name__ == "__main__":

    signature_test = {"user_nickname":["wo多伦多是往返$"],"signature":["我来测试一下，看看好用不房产？"]}
    df_name_sig =  pd.DataFrame(signature_test)
    df_name_sig['signature'] = df_name_sig['signature'].fillna(0)
    df_string = apply_and_concat(df_name_sig, 'signature', string_len, ['count_en', 'count_dg', 'count_sp', 'count_zh', 'count_pu'])
    df_string['ads_status'] = df_name_sig['signature'].apply(ads_search)
    df_string['phone_status'] = df_name_sig['signature'].apply(phone_search)
    df_string['phone_status'] = df_name_sig['signature'].apply(phone_search)
    df_string = apply_and_concat(df_string, 'signature', count_words, ['kind_words', 'num_words'])
    
    df_string['name_status'] = df_name_sig['user_nickname'].apply(ads_search)
    df_string['name_phone'] = df_name_sig['user_nickname'].apply(phone_search)
    df_string.drop(['user_nickname','signature'],axis=1,inplace=True)
    x_test = df_string.drop(['count_pu','count_dg','count_zh'],axis=1)
    # predict users' behaviour
    model = joblib.load('machinelearning_model_svm.pkl')
    P = model.predict_proba(x_test)
    # output = ['Probability of ads' 'Prob of not ads']
    print("Probability of releasing ADS is: %s"% P[0][0])
    

Probability of releasing ADS is: 0.9568067744791658
